# 🚀 Generative AI for Developers (C#)
`Author: Arafat Tehsin`

Well, well, well, looks like we have a language polyglot in the making! And not just any polyglot, but one who's delving into the fascinating world of Azure OpenAI Service. Get ready to unleash your inner coding genius as we take you on a journey from GPT-3 Text Completions to ChatGPT (yes, we're getting chatty here!) and even sprinkle in some DALL-E for good measure. Buckle up, folks, because we're about to turn up the AI heat!

---

```
Inspired by the great designers at the Microsoft OCTO (Semantic Kernel)
```
### Reminder: This 📘 `Polyglot` notebook needs to be run from VS Code with below pre-requisites. 

The following software needs to be installed on your computer:

* ✅ [Visual Studio Code](https://code.visualstudio.com/Download)
* ✅ The latest [.Net 7.0 SDK](https://dotnet.microsoft.com/en-us/download) 
* ✅ [Polyglot Notebooks for VS Code](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode?)

When the three items above are installed, the code notebooks should be formatted nicely with code blocks that have a little ▶️ (play) button next to them when you hover over the code. The "Polyglot Notebooks for VS Code" extension is what lets you run all the code snippets from within VS Code and serves as a little playground for your initial Azure OpenAI learning.

Apart from installing software, you will need to have an API key to access the OpenAI models. 

* ✅ [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?WT.mc_id=AI-MVP-5003464). Access your API key on Azure Open AI Service with [these instructions](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference?WT.mc_id=AI-MVP-5003464).

This notebook also requires you to create a deployment of [models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models?WT.mc_id=AI-MVP-5003464) 🧪. This means that your fresh Azure OpenAI Service won't have any model right now. 
* ✅ You can create your deployments from [these docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal&WT.mc_id=AI-MVP-5003464). The suggestion is to have the deployments similar to their model name. This way OpenAI API would work without any changes in the code:

    - `text-davinci-002` - davinci002
    - `gpt-35-turbo-16k` - gpt35turbo16k
    -  `gpt-4`- gpt40613



### 🌱 Environment Variables

Once you get the key and enpoint, you may want to setup from command line or the shiny [Windows Terminal](https://aka.ms/windowsterminal). Again, you can keep any environment variable but this is an example. Mine would be a little different than this too, considering I am working on different Azure regions of Azure OpenAI Service.

* `setx AZUREOPENAI_KEY "your_key_here"`
* `setx AZUREOPENAI_ENDPOINT "your_enpoint_here"`

Or if you want to use OpenAI

* `setx OPENAI_KEY "your key here"`

#### 📘 After you have achieved all the 👆 prerequisites, open the notebooks in this repo from within Visual Studio Code and you are ready to to go!
---

### Now, let's try this out

👋 In the event that you hit "play" below on the code block and it asks you to:
 
```
Select kernel for <filename>
----
.NET Interactive 👈
Select Another Kernel...
----
```

Choose `.NET Interactive` and you're good to go. That selection lets you magically run the Generative AI notebooks.

In [ ]:
// 👈 you should see a ▶️ (play) button on 
// the left if you click in this block, 
// or just hover over this block
// CLICK IT! And you should see an output
// below this code block.

Console.WriteLine("Microphone test. Check one. Two. Three.");

## Get a 🔥 .NET Interactive kernel ready for you to load the Azure OpenAI Service package

This will create a client so you don't have to initialise it again. You can read more about the [Azure OpenAI SDK](https://www.nuget.org/packages/Azure.AI.OpenAI/) here.

In [ ]:
// Getting the latest package released on 15th Dec, 2023 - https://www.nuget.org/packages/Azure.AI.OpenAI
#r "nuget:Azure.AI.OpenAI,*-*"

using Azure;
using Azure.AI.OpenAI;
using static System.Environment;

bool useAzureOpenAI = true;

// setting constant for models
const string davinci002 = "davinci-002";
const string gpt35turbo16k = "gpt-35-turbo-16k";
const string gpt40613 = "gpt-4-default";


string azureOpenAIKey = Environment.GetEnvironmentVariable("AOI_KEY_SWDN"); // I have got a few 
string azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AOI_ENDPOINT_SWDN");
string openaiKey = Environment.GetEnvironmentVariable("OPENAI_KEY");
string cognitiveSearchKey = Environment.GetEnvironmentVariable("ENTERPRISEGPT_COGNITIVESEARCH_KEY");

OpenAIClient client = useAzureOpenAI
    ? new OpenAIClient(
        new Uri(azureOpenAIEndpoint),
        new AzureKeyCredential(azureOpenAIKey))
    : new OpenAIClient(openaiKey);

If you can see a confirmation line above that starts with `Installed Packages ... • Azure.AI.OpenAI, 1.0.0-beta.8, ...` then proceed below.

## 📑 Let's talk about completion with GPT-3

Completions can be used for variety of tasks. It's a simple but powerful text-in and text-out interface. 

In this snippet, we will be setting up our respective engine for GPT-3 which is `davinci` and also a `prompt` which we want to see completed by GPT-3 model.

Upon providing your input as a prompt, the model will generate a text completion that attempts to match whatever context or pattern you gave it. For example, if you a prompt, "
All that glitters is", it will return the completion " not gold" with high probability.

💡 Read more about the Completions [here](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/completions?WT.mc_id=AI-MVP-5003464).

In [ ]:
string prompt = "I was going to the airport and I saw";
Console.Write($"Your input: {prompt}");

CompletionsOptions completionsOptions = new()
{
    DeploymentName = davinci002, 
    Prompts = { prompt },
    MaxTokens = 40,
    Temperature = 0.9F,   
};

Response<Completions> completionsResponse = client.GetCompletions(completionsOptions);
string completion = completionsResponse.Value.Choices[0].Text;
Console.Write($"{completion}");

## 📈 Sentiment Classifier with Completion API

In this example, we provide some sample utterances (past records) to augment the model according to our needs. These examples are a few and hence a good example for few-shot learning within Prompt Engineering.

In [ ]:
string prompt = """
This is a tweet sentiment classifier. Just provide a sentiment either Positive or Negative. Below is the data

---------------------
DATA
---------------------

Tweet: "I loved the new Batman movie!"
Sentiment: Positive

Tweet: "I hate it when my phone battery dies." 
Sentiment: Negative

Tweet: "My day has been 👍"
Sentiment: Positive

Tweet: "This is the link to the article"
Sentiment: Neutral

Tweet: "I just love this cake."
Sentiment: 
""";
Console.WriteLine($"Your input: {prompt}");

CompletionsOptions completionsOptions = new()
{
    DeploymentName = davinci002, 
    Prompts = { prompt },
    MaxTokens = 10,
    Temperature = 0.9F,
    StopSequences = { "\n" }
};

Response<Completions> completionsResponse = client.GetCompletions(completionsOptions);
string completion = completionsResponse.Value.Choices[0].Text;
Console.Write($"GPT-3 Response (Chat): {completion.Trim()}");

## 💬 ChatGPT & GPT-4

Previous models were text-in and text-out, meaning they accepted a prompt string and returned a completion to append to the prompt. However, the ChatGPT and GPT-4 models are conversation-in and message-out. The models expect input formatted in a specific chat-like transcript format, and return a completion that represents a model-written message in the chat. 

In this case, we're trying to build a flight tracker with the few-shot examples. 

* ✅ System message, the first one with `ChatRole.System` key is a base for your conversational experience. Try to write as much as you want within it. 
* ✅ Also, the subsequent messages with are the few shots. `ChatRole.User` defines what user may say whereas `ChatRole.Assisant` responds back with the probable answer.

You can also use the similar approach to build other use-cases. 

💡 Learn more about the ChatGPT & GPT-4 models [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions&WT.mc_id=AI-MVP-5003464).

In [ ]:
// Enter your query here
var prompt = "I want to track my flight from Bali";

var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        new ChatRequestSystemMessage("You are a virtual flight assistant who is responsible for tracking flights. You need either a flight number or source (to) and destination (from) to track any flight. If user does not provide either of this information then you should ask for it. Anything outside of the flights are not a part of your scope and you should just answer with an Unknown intent."),
        new ChatRequestUserMessage("Track my flight from Sydney?"),
        new ChatRequestAssistantMessage("Sure! Please provide your destination?"),
        new ChatRequestUserMessage("I want to track flight JF-123?"),
        new ChatRequestAssistantMessage("Your flight JF-123 is now departed from Sydney and will be arriving soon in Melbourne at 7 PM"),
        new ChatRequestUserMessage(prompt)
    },
    MaxTokens = 100,
    DeploymentName = gpt35turbo16k
};

await foreach (StreamingChatCompletionsUpdate chatUpdate in client.GetChatCompletionsStreaming(chatCompletionsOptions))
{
    if (chatUpdate.Role.HasValue)
    {
        Console.Write($"{chatUpdate.Role.Value.ToString().ToUpperInvariant()}: ");
    }
    if (!string.IsNullOrEmpty(chatUpdate.ContentUpdate))
    {
        Console.Write(chatUpdate.ContentUpdate);
    }
}

`🔔 You will be able to look at the multi-turn chat application and that too with voice, very soon in this repo.`

## 🧠 Summarise and Translate 

While this format was designed specifically for multi-turn conversations, you'll find it can also work well for non-chat scenarios too.

In [ ]:
string textToSummarizeAndTranslate = @"
   As you know, Microsoft’s AI Builder now supports prebuilt Azure OpenAI models, allowing users to easily integrate powerful AI capabilities into their applications and processes without the need for any coding or data science expertise. In this post, we will explore the benefits and potential use cases of leveraging these prebuilt models with AI Builder.


Microsoft’s AI Builder simplifies the integration of Azure OpenAI models by providing a user-friendly interface for connecting to the OpenAI API. This enables users to access advanced AI capabilities, such as GPT and incorporate them into their applications and workflows with minimal effort. By utilizing prebuilt Azure OpenAI models, users can add powerful AI-driven features to their applications and processes, such as natural language processing, text generation, and more. This can lead to improved efficiency, better decision-making, and more personalized user experiences.

Helpful conversational experiences with Power Virtual Agents
These latest capabilities of Azure OpenAI in AI Builder are native to Power Automate and Power Apps. This basically means that anything in Power Platform can utilise the capabilities of the GPT models without going to external sources. As a result of that, I created a Power Virutal Agents (PVA) bot using the unified authoring canvas which calls a Power Automate flow to get the answers from Azure OpenAI‘s GPT model.";

string summarizationPrompt = @$"
    Summarize the following text for a 140 character tweet and translate it into Spanish.

    Text:
    """"""
    {textToSummarizeAndTranslate}
    """"""

    Summary:
";

// Console.Write($"Input: {summarizationPrompt}");

var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        new ChatRequestSystemMessage("You're an AI Virtual Assistant responsible for the summarisation and translating the text."),
        new ChatRequestUserMessage(summarizationPrompt)
    },
    MaxTokens = 200,
    DeploymentName = gpt40613
};

await foreach (StreamingChatCompletionsUpdate chatUpdate in client.GetChatCompletionsStreaming(chatCompletionsOptions))
{
    if (chatUpdate.Role.HasValue)
    {
        Console.Write($"{chatUpdate.Role.Value.ToString().ToUpperInvariant()}: ");
    }
    if (!string.IsNullOrEmpty(chatUpdate.ContentUpdate))
    {
        Console.Write(chatUpdate.ContentUpdate);
    }
}

## 🖼️ Generate an image with DALL-E 2

The image generation API creates an image from a text prompt. It does not edit existing images or create variations. 

#### 🛖 But first, a little housekeeping

You may notice a long piece of code and it has a couple of reasons:

* ⚠️ The Azure.OpenAI SDK does not have a support for DALL-E yet.
* 🫰 In order to show you an image on this Notebook, there's a way which moved Qasim   



#### 🎨 Change the prompt and see the magic! 

In [ ]:
#r "nuget: SkiaSharp, 2.88.3"

#!import Utilities.cs

Response<ImageGenerations> imageGenerations = await client.GetImageGenerationsAsync(
    new ImageGenerationOptions()
    {
        Prompt = "People carrying umbrella on a hot summer day",
        Size = ImageSize.Size1024x1024,
        DeploymentName = "dall-e-3"
    });

// Image Generations responses provide URLs you can use to retrieve requested images
Uri url = imageGenerations.Value.Data[0].Url;
    
await Utilities.ShowImage(url.ToString(), 512, 512);


## 🏢 ChatGPT for your Enterprise with Azure OpenAI on your data

Azure OpenAI on your data is a feature that allows you to use OpenAI’s powerful language models (like GPT-35-Turbo or GPT-4) to generate responses based on your own data. This is done through a REST API or a web-based interface in Azure OpenAI Studio.

Think of it like this: You have a database (your box of toys), and you want to find specific information (your favorite toy). Azure OpenAI on your data (your super smart friend) helps you find that information quickly and efficiently.

It’s like having a super smart friend who not only knows everything in your database but can also find and tell you exactly what you want to know in seconds! And the best part? You don’t need to write complex queries or sift through tons of data yourself.

Before you start, make sure you have been approved for Azure OpenAI access and have an Azure OpenAI Service resource with either the gpt-35-turbo or the gpt-4 models deployed. Happy coding! 😊

In [ ]:
AzureCognitiveSearchChatExtensionConfiguration contosoExtensionConfig = new()
{
    SearchEndpoint = new Uri("https://your-contoso-search-resource.search.windows.net"),
    Authentication = new OnYourDataApiKeyAuthenticationOptions("<your Cognitive Search resource API key>"),
    IndexName = "your-index-name"
};

ChatCompletionsOptions chatCompletionsOptions = new()
{
    DeploymentName = "gpt-35-turbo-0613",
    Messages =
    {
        new ChatRequestSystemMessage(
            "You are a helpful assistant that answers questions about the Contoso product database."),
        new ChatRequestUserMessage("What are the best-selling Contoso products this month?")
    },

    // The addition of AzureChatExtensionsOptions enables the use of Azure OpenAI capabilities that add to
    // the behavior of Chat Completions, here the "using your own data" feature to supplement the context
    // with information from an Azure Cognitive Search resource with documents that have been indexed.
    AzureExtensionsOptions = new AzureChatExtensionsOptions()
    {
        Extensions = { contosoExtensionConfig }
    }
};

Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);
ChatResponseMessage message = response.Value.Choices[0].Message;

// The final, data-informed response still appears in the ChatMessages as usual
Console.WriteLine($"{message.Role}: {message.Content}");

// Responses that used extensions will also have Context information that includes special Tool messages
// to explain extension activity and provide supplemental information like citations.
Console.WriteLine($"Citations and other information:");

foreach (ChatResponseMessage contextMessage in message.AzureExtensionsContext.Messages)
{
    // Note: citations and other extension payloads from the "tool" role are often encoded JSON documents
    // and need to be parsed as such; that step is omitted here for brevity.
    Console.WriteLine($"{contextMessage.Role}: {contextMessage.Content}");
}

## ⚡ Function Calling (Coming soon)

When you code, sometimes you need to do tasks that are quite common and have been done by many others before, like sorting a list of numbers, finding the current date and time or even complex tasks like making a network request to fetch some data from the internet. Instead of writing all the code for these tasks yourself, you can use pre-written code in the form of functions that come with programming languages or are available in libraries.

These functions are like little machines: you give them some input, they do something with that input and then they give you back some output. For example, a function to calculate the square of a number would take a number as input, multiply it by itself and then give you the result.

OpenAI’s function calling is similar. OpenAI has defined a set of functions that the AI can call to perform specific tasks. These tasks could be anything from searching the web to generating an image based on a description. When the AI “calls” one of these functions, it provides the necessary input and then receives the output once the function has completed its task.

Just like when you’re coding, these functions help the AI to perform tasks more efficiently and effectively, without needing to figure out how to do everything from scratch.

### ✈️ Optimising the weather operations 

In our previous example, we saw that how we shaped up the prompts to get the respective responses while enforcing the required values in our flight tracker. In the below example, we will utilise the function calling capability to optimise our operations. 


In [ ]:
// Coming soon.